Step 1:- Import Required Libraries

In [ ]:
!pip uninstall -y huggingface_hub

In [ ]:
!pip install -U huggingface_hub>=0.35.0

In [ ]:
Step 2:-Hugging face Authentication

In [ ]:
from huggingface_hub import login
login()

Step 3 :-Verify who

In [ ]:
from huggingface_hub import whoami
whoami()

Step 4:- Load LLaMA-2 7B Chat Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Install required libraries for 8-bit quantization
!pip install bitsandbytes accelerate

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define quantization config
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config, # Pass quantization config here
    dtype=torch.float16
)

model.eval()

Step 5:-Conversation Prompt Formatter

In [ ]:
def format_prompt(system_prompt, conversation):
    prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n"
    for user, assistant in conversation:
        prompt += f"{user} [/INST] {assistant} </s><s>[INST] "
    return prompt

Step 6:-Chatbot Response Function

In [ ]:
def chatbot_reply(user_input, conversation):
    system_prompt = "You are a helpful, polite, and knowledgeable assistant."

    conversation.append((user_input, ""))
    prompt = format_prompt(system_prompt, conversation)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    assistant_reply = decoded.split(user_input)[-1].strip()

    conversation[-1] = (user_input, assistant_reply)
    return assistant_reply


Step 7:-Make chat Interactive /Conversational

In [ ]:
conversation = []

print("🤖 Conversational Chatbot! Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    response = chatbot_reply(user_input, conversation)
    print(f"\nBot: {response}\n")